In [2]:
import polars as pl

# Load the raw data (only rows with date_id >= 85)
train_data = pl.read_parquet('../data/jane-street/train.parquet').filter(pl.col('date_id') >= 85)

# Display the first few rows
train_data.head()


date_id,time_id,symbol_id,weight,feature_00,feature_01,feature_02,feature_03,feature_04,feature_05,feature_06,feature_07,feature_08,feature_09,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31,feature_32,…,feature_52,feature_53,feature_54,feature_55,feature_56,feature_57,feature_58,feature_59,feature_60,feature_61,feature_62,feature_63,feature_64,feature_65,feature_66,feature_67,feature_68,feature_69,feature_70,feature_71,feature_72,feature_73,feature_74,feature_75,feature_76,feature_77,feature_78,responder_0,responder_1,responder_2,responder_3,responder_4,responder_5,responder_6,responder_7,responder_8,partition_id
i16,i16,i8,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,i8,i8,i16,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,…,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,i64
85,0,0,3.109312,null,null,null,null,null,-0.450499,-1.616381,-2.013025,-0.017599,11,7,76,-1.077771,0.039866,-0.65978,null,-0.261007,null,-1.936319,-1.190953,0.696218,null,0.484168,0.294946,-0.96987,0.160298,null,null,0.879279,-0.541101,-0.214313,null,null,…,null,null,-1.164152,null,-2.935171,1.012571,null,-3.249436,-0.887255,-0.869813,null,null,null,null,null,-0.951475,0.230537,-0.596571,-0.856555,-0.195387,-0.671425,null,null,-0.281401,-0.210105,-0.246561,-0.286114,0.62552,-0.310852,-0.049727,0.534198,0.198891,-1.069264,0.08607,0.273769,-1.012277,0
85,0,1,4.173016,null,null,null,null,null,-0.561003,-2.013899,-1.988785,-0.028133,11,7,76,-0.89596,0.358419,-0.424778,null,-0.096455,null,-0.653376,-1.457888,1.792939,null,2.419092,1.158298,-1.14655,0.881267,null,null,1.163396,-0.303266,-0.182955,null,null,…,null,null,-2.188706,null,-2.328486,0.458066,null,-1.753266,-1.392433,-0.869813,0.688687,0.222187,1.374588,-0.990737,-1.071639,-0.788221,0.49704,-0.285493,-0.868851,-0.05351,-0.686293,null,null,-0.207581,-0.246351,-0.27867,-0.302826,0.503355,-0.094774,1.376147,0.487139,-0.026323,1.262791,0.233934,0.022418,0.401515,0
85,0,2,1.381817,null,null,null,null,null,-0.7775,-2.553066,-3.040147,-0.03267,81,2,59,-0.566227,5.327168,1.528497,null,-0.112716,null,-1.198659,-1.770741,-0.638912,null,-0.356265,-0.464235,-1.340848,-1.365498,null,null,0.441449,-0.586902,-0.079118,null,null,…,null,null,-0.672283,null,-2.163897,0.789618,null,-2.023751,-1.206249,-0.869813,null,null,null,null,null,-0.480621,5.894732,1.228865,-0.608697,4.075598,1.069681,null,null,-0.184736,-0.345173,-0.151796,-0.264521,5.0,5.0,-1.982797,5.0,5.0,-0.205135,0.4033,0.793304,2.807026,0
85,0,7,2.106224,null,null,null,null,null,-0.995678,-1.619547,-2.39677,-0.021119,11,7,76,-1.318322,0.047269,-0.751898,null,1.456272,null,-1.3534,-1.394393,3.133692,null,0.178299,0.327529,-1.510156,-1.147885,null,null,-0.210504,0.401724,1.20255,null,null,…,null,null,-2.516531,null,-1.294529,1.0471,null,-2.190469,-1.514988,-0.869813,null,null,null,null,null,-1.055661,0.463701,-0.682089,-1.007249,-0.312894,-1.017619,null,null,-0.276763,-0.255561,-0.328656,-0.257443,-0.781161,-0.631942,0.152099,-2.374984,-1.785865,-0.273755,-1.696202,-0.548582,-0.81586,0
85,0,8,1.707147,null,null,null,null,null,-0.528211,-1.554091,-1.63106,-0.008948,15,1,62,1.293916,18.171343,7.267656,null,0.585029,null,-1.528019,-3.131001,-0.502737,null,-0.314039,0.481302,-1.077692,-0.561491,null,null,-1.558967,0.12489,0.720276,null,null,…,null,null,-2.123032,null,-0.706382,1.002747,null,-2.344671,-1.567466,-0.869813,null,3.649192,2.833574,null,null,0.606422,28.107288,3.738327,0.795478,16.122564,6.776067,null,null,-0.22111,-0.286459,-0.292706,-0.256621,4.280499,4.502849,1.735608,4.230651,5.0,-0.107674,1.533022,5.0,-4.17551,0


In [3]:
# Define columns to exclude
exclude_columns = ['date_id', 'time_id', 'symbol_id', 'weight', 'partition_id'] + \
                  [col for col in train_data.columns if col.startswith('responder_')]

# Select feature columns
feature_columns = [col for col in train_data.columns if col not in exclude_columns]


In [ ]:
# Generate lagged features for numerical feature columns
for feature in feature_columns:
    train_data = train_data.with_columns(
        pl.col(feature).shift(1).over("symbol_id").alias(f"{feature}_lag_1")
    )


In [ ]:
# Generate difference features
for feature in feature_columns:
    train_data = train_data.with_columns(
        (pl.col(feature) - pl.col(f'{feature}_lag_1')).alias(f'{feature}_lag_diff_1')
    )


In [ ]:
# Generate ratio features
for feature in feature_columns:
    train_data = train_data.with_columns(
        (pl.col(feature) / (pl.col(f'{feature}_lag_1') + 1e-9)).alias(f'{feature}_lag_ratio_1')
    )


In [ ]:
# Fill NaN values generated by lagging
train_data = train_data.fill_null(0)


In [ ]:
# Save the dataset for XGBoost (raw features + engineered features, no scaling)
train_data.write_parquet('../data/XGBoost_Featured.parquet', compression='snappy')


In [ ]:
from sklearn.preprocessing import MinMaxScaler

# Extract feature columns as NumPy for scaling
scaler = MinMaxScaler()
scaled_features = scaler.fit_transform(train_data.select(feature_columns).to_numpy())

# Replace scaled columns in the Polars DataFrame
scaled_df = pl.DataFrame(scaled_features, schema=feature_columns)
train_data = train_data.drop(feature_columns).hstack(scaled_df)

# Save the scaled dataset for TFT
train_data.write_parquet('../data/TFT_Featured_Scaled.parquet', compression='snappy')
